# get similarity scores for a list of cases

Alternative for get_similarities() from similiarity_matrix.py

Uses closure for workhorse function.

In [ ]:

def make_get_similarity(similarity_matrix, CLid_to_index):
    """
    helper funciton for get_similarities
    """
    def f(CLid_pair):
        try:
            ida = CLid_to_index[CLid_pair[0]]
            idb = CLid_to_index[CLid_pair[1]]
        
            return similarity_matrix[ida, idb]
        except KeyError:
            return np.nan
        
    return f
    
def get_similarities(similarity_matrix, CLid_pair, CLid_to_index):
    """
    Returns the similarities for cases index by CL ids as a list

    Parameters
    ----------
    similarity_matrix: precomputed similarity matrix

    CLid_A, CLid_B: two lists of CL ids whose similarities we want

    CLid_to_index: dict that maps CL ids to similarity_matrix indices
    """
    get_similarity = make_get_similarity(similarity_matrix, CLid_to_index)
    return [get_similarity( pair) for pair in CLid_pair]

# sample absent edges

In [ ]:
# this ended up being a lot slower than sampling an arbitrary pair of vertices then checking the conditions
def sample_absent_edges(G, num_samples, active_years, seed=None):
    if seed:
        random.seed(seed)
        
    # citing cases must be in active_years
    possible_ing_cases = set(G.vs.select(year_ge = min(active_years), year_le = max(active_years)))

    samples = set()
    present_edges = set(G.get_edgelist()) 
    
    while len(samples) < num_samples:
        # sample a citing case
        ing_vertex = random.sample(possible_ing_cases, 1)[0]
        
        # get possible cited cases
        # cited cases must be strictly before citing case
        possible_ed_cases = G.vs.select(year_le = ing_vertex['year'] - 1)
        
        # sample a cited case
        ed_vertex = random.sample(possible_ed_cases, 1)[0]
        
        absent_edge = (ing_vertex.index, ed_vertex.index)
        
        if (absent_edge not in present_edges) and (absent_edge not in samples):
            samples.add(absent_edge)
            
    return samples